In [2]:
import transformers
import datasets
import pandas as pd
import numpy as np

In [3]:
model_checkpoint = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-xsmall"

In [4]:
from datasets import Dataset


In [5]:
df = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
df.head(10)




,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
5,Cell phone use should not be legal while drivi...,0,Phones and driving,persuade_corpus,False
6,Phones and Driving\n\nDriving is a good way to...,0,Phones and driving,persuade_corpus,False
7,PHONES AND DRIVING\n\nIn this world in which w...,0,Phones and driving,persuade_corpus,False
8,People are debating whether if drivers should ...,0,Phones and driving,persuade_corpus,False
9,Texting and driving\n\nOver half of drivers in...,0,Phones and driving,persuade_corpus,False


In [6]:
train = df[df.prompt_name != 'Car-free cities'].reset_index(drop=True)
valid = df[df.prompt_name == 'Car-free cities'].reset_index(drop=True)
train.head()

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False


In [7]:
not_persuade_df = train[train['source'] != 'persuade_corpus']
persuade_df = train[train['source'] == 'persuade_corpus']
sampled_persuade_df = persuade_df.sample(n=6000, random_state=42)

In [8]:
all_human = set(list(''.join(sampled_persuade_df.text.to_list())))
other = set(list(''.join(not_persuade_df.text.to_list())))

In [9]:
chars_to_remove = ''.join([x for x in other if x not in all_human])
print(chars_to_remove)

ま╯📚í🚣🧹🍝😬🚪🎅️ê💆🚕😎🤖🏡🍲🍽🏨🎭🌎📝注🐱É。🛬🏢ん🔋🤦🐻🇷🥖□ó驾🎈​在🔬^🇸ã🎓🍁🌸🌳🏃🧚😆♂–择🧬🏙🧦💼🤪🎣🐝é🏈💁😡🌃🥟🙅💰影🐾👋🏊机ç🦁🧭🍎🚨🍓💔🤣😠💪😮😵🥨📺“🥔🤜💤💅😋🤓🇧🥕🤘に💨🎹🎵🍔🚚😖😁あ🎧🙌🐴🕒🇪🌭😳☀🔥🙄🥁🧖🥶🥳驶🏖🦎🌨禁👩🍜👏将🎠み😒👕保🥗🌈🐕🛸÷😉🚌🎃🙏🌽🏼👬🎨都📹🐸🕺🍮🐭🤔🔑完须✨🇯响😜集🙈🏠🎯😂力”😲🐶😔🦄🍟🍰🥤💚📉🎄📖🇺👨🔭🏄😩💃🌧🥭🐳🌷司😹😊‍📞💻🌏🎬🎶🎢💸🤟🛋👻📦使一合🏳💉ü法🏫🏯🚀🛫と🎩🍄🍞🥪🇫手🍭す😷🔜🍣🤛🦸取💬😅所🇵🍿🔍🏰🌯中😴👍🥯🍋😈🚂🚭💦🐰🕹🤤道🛀😝🎤🥩📈🗣🌊🧽☹上💫🍖👧🐒🌻🧡🤞🙊👌💥🌠うは📅😌🌲⏰


In [10]:
not_persuade_df.loc[:, 'text'] = not_persuade_df['text'].fillna('')

translation_table = str.maketrans('', '', chars_to_remove)
def remove_chars(s):
    return s.translate(translation_table)
not_persuade_df.loc[:, 'text'] = not_persuade_df['text'].apply(remove_chars)


In [11]:
train = pd.concat([not_persuade_df, sampled_persuade_df]).sample(frac=1, random_state=42).reset_index(drop=True)
train.source.value_counts()

source
persuade_corpus                       6000
llama2_chat                           2411
chat_gpt_moth                         2409
mistral7binstruct_v1                  2408
mistral7binstruct_v2                  2406
llama_70b_v1                           984
darragh_claude_v6                      952
darragh_claude_v7                      951
falcon_180b_v1                         899
kingki19_palm                          672
train_essays                           670
cohere-command                         301
palm-text-bison1                       300
radek_500                              250
mistralai/Mistral-7B-Instruct-v0.1     201
NousResearch/Llama-2-7b-chat-hf        200
radekgpt4                              100
Name: count, dtype: int64

In [12]:
ds_train = Dataset.from_pandas(train)
ds_valid = Dataset.from_pandas(valid)

In [13]:
ds_train


Dataset({
    features: ['text', 'label', 'prompt_name', 'source', 'RDizzl3_seven'],
    num_rows: 22114
})

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [15]:
def preprocess_function(examples):
    return tokenizer(examples['text'], max_length=128, padding=True, truncation=True)

In [16]:
ds_train_enc = ds_train.map(preprocess_function, batched=True)


  0%|          | 0/23 [00:00<?, ?ba/s]

In [17]:
ds_valid_enc = ds_valid.map(preprocess_function, batched=True)


  0%|          | 0/5 [00:00<?, ?ba/s]

In [18]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

2024-03-21 16:01:06.715666: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 16:01:06.715819: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 16:01:06.840867: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.

In [19]:
metric_name = "roc_auc"
model_name = "deberta-xsmall"
train_batch_size = 4
eval_batch_size = 32
grad_acc = 4

In [20]:
num_steps = len(train) // (train_batch_size * grad_acc)
num_steps

1382

In [21]:
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = num_steps // 3,
    save_steps = num_steps // 3,
    learning_rate=2e-5,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    gradient_accumulation_steps=grad_acc,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
    report_to='none', # change to wandb after enabling internet access
)

In [22]:
from sklearn.metrics import roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    auc = roc_auc_score(labels, probs[:,1], multi_class='ovr')
    return {"roc_auc": auc}

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_train_enc,
    eval_dataset=ds_valid_enc,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
trainer.train()


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Roc Auc
460,No log,0.199364,0.998508


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=691, training_loss=0.09557693359993301, metrics={'train_runtime': 508.5887, 'train_samples_per_second': 43.481, 'train_steps_per_second': 1.359, 'total_flos': 364157494247424.0, 'train_loss': 0.09557693359993301, 'epoch': 1.0})

In [28]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
test_ds = Dataset.from_pandas(test)
test_ds_enc = test_ds.map(preprocess_function, batched=True)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [29]:
test_preds = trainer.predict(test_ds_enc)


In [30]:
logits = test_preds.predictions
probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
sub = pd.DataFrame()
sub['id'] = test['id']
sub['generated'] = probs[:,1]
sub.to_csv('submission.csv', index=False)
sub.head()

,id,generated
0,0000aaaa,0.259643
1,1111bbbb,0.986910
2,2222cccc,0.990840


In [31]:
res = []
for src in valid.source.unique():
    if src in ['train_essays', 'persuade_corpus', 'original_moth']: continue
    test2  = valid[valid['source'].isin([src, 'train_essays'])]
    test_ds2 = Dataset.from_pandas(test2)
    test_ds_enc2 = test_ds2.map(preprocess_function, batched=True)
    eval_result = trainer.evaluate(test_ds_enc2)
    score = eval_result['eval_roc_auc']
    res.append(f'{src}: {score}')
    
for r in res: print(r)

  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/1 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/2 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


mistral7binstruct_v2: 0.9756011315417256
llama_70b_v1: 0.9952365984897809
darragh_claude_v7: 0.9918811881188119
radek_500: 0.9974134579081129
darragh_claude_v6: 0.9916577663597264
falcon_180b_v1: 0.9951260822169569
mistral7binstruct_v1: 0.9722166094160436
chat_gpt_moth: 0.9704058317919704
llama2_chat: 0.9654108267969653
NousResearch/Llama-2-7b-chat-hf: 0.9957180152983315
mistralai/Mistral-7B-Instruct-v0.1: 0.9968670438472418
palm-text-bison1: 0.9916548797736916
cohere-command: 0.9916265912305515
radekgpt4: 0.9946713907600094
kingki19_palm: 0.9989942292165501
